In [ ]:
#필요한 라이브러리 호출 공간
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

데이터 전처리

In [ ]:
# 데이터 불러오기
file_name = ""
def reading_csv(filename):
    df = pd.read_csv(filename)
    print(df.shape)
    print(df.head())
    return df

df = reading_csv(file_name)

# 결측치 확인
print(df.isnull().sum())

# 결측치 처리 (평균 or 최빈값 or 중위값)
si = SimpleImputer(strategy="mean")
ss = StandardScaler()
le = LabelEncoder()
oe = OneHotEncoder(sparse=False)
num_col = []
cat_col = []
 
def processing(df):
    df = df.copy()

    # 수치형 컬럼 처리
    df[num_col] = si.fit_transform(df[num_col])
    df['파생변수'] = pd.cut(df['col'], bins=[0, 18, 30, 50, 100], labels=['Teen', 'Young', 'Adults', 'Senior'])
    df[num_col] = ss.fit_transform(df[num_col])

    # 범주형 컬럼 처리
    for col in cat_col:# label encoding
            df[col] = le.fit_transform(df[col])

    return df

processed_df = processing(df)
print(processed_df.head())


Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# 타겟 변수 지정
target_col = 'target'  # 실제 변수명으로 변경

X = df.drop(columns=[target_col])
y = df[target_col]

# 훈련/검증 데이터 분할 (8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


모델 적용

In [ ]:
# 모델 적용 및 성능 평가
from sklearn import linear_model  
# 로지스틱 회귀 (이진 분류에 특화)
from sklearn.linear_model import LogisticRegression  
# 앙상블 모델: 여러 결정트리를 결합하여 성능 향상
from sklearn.ensemble import RandomForestClassifier  
# 단순 신경망 기반의 선형 분류기
from sklearn.linear_model import Perceptron  
# 확률적 경사하강법 기반 선형 분류기
from sklearn.linear_model import SGDClassifier  
# 결정 트리 기반 분류기 (if-else 형태의 트리 구조)
from sklearn.tree import DecisionTreeClassifier  
# K-최근접 이웃 알고리즘 (거리 기반 분류)
from sklearn.neighbors import KNeighborsClassifier  
# 서포트 벡터 머신 (복잡한 경계 분류, 마진 기반)
# LinearSVC: 선형 SVM (SVM의 일종, 대용량에 빠르고 선형 분리에 적합)
from sklearn.svm import SVC, LinearSVC  
# 확률 기반 분류기 (조건부 독립 가정)
from sklearn.naive_bayes import GaussianNB  

# 🔹 [3] 시각화 도구 임포트
import seaborn as sns  
%matplotlib inline  
from matplotlib import pyplot as plt  
from matplotlib import style  # 그래프 스타일 지정용

# 의사결정 나무 --> 복자도가 높아지면 일반화 성능이 떨어짐, 과적합 가능성
decision_tree=DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred=decision_tree.predict(X_test)
train_acc_decision_tree=round(decision_tree.score(X_train, y_train)*100, 2)
test_acc_decision_tree=round(decision_tree.score(X_test, y_test)*100, 2)
train_acc_decision_tree,test_acc_decision_tree


# 랜덤 포레스트 --> 과적합 방지에 강함, 하지만 하이퍼파라미터 기본값 사용 등으로 일반화 성능이 크게 향상되지 않음
# max_depth, min_sample_leaf 제한
# n_estimator 증가
# GridSearchCV 사용 --> 최적의 하이퍼 파라미터 조건 탐색
random_forest=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_prediction=random_forest.predict(X_test)
random_forest.score(X_train, y_train)
train_acc_random_forest=round(random_forest.score(X_train, y_train)*100, 2)
test_acc_random_forest=round(random_forest.score(X_test, y_test)*100, 2)
train_acc_random_forest,test_acc_random_forest


# 로지스틱 회귀 --> 과적합 위험 낮음, 특히 특성이 별로 없을 때
log_reg=LogisticRegression()
log_reg.fit(X_train, y_train)
Y_pred=log_reg.predict(X_test)
train_acc_log=round(log_reg.score(X_train, y_train)*100, 2)
test_acc_log=round(log_reg.score(X_test, y_test)*100, 2)
train_acc_log, test_acc_log


# 나이브 베이즈 --> 로지스틱 회귀보다 성능 낮음
gaussian=GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred=gaussian.predict(X_test)
train_acc_gaussian=round(gaussian.score(X_train, y_train)*100, 2)
test_acc_gaussian=round(gaussian.score(X_test, y_test)*100, 2)
train_acc_gaussian,test_acc_gaussian

# Linear SVC
svc=LinearSVC()
svc.fit(X_train, y_train)
Y_pred=svc.predict(X_test)
train_acc_svc=round(svc.score(X_train, y_train)*100, 2)
test_acc_svc=round(svc.score(X_test, y_test)*100, 2)
train_acc_svc,test_acc_svc

# Perceptron
perceptron=Perceptron(max_iter=5)
perceptron.fit(X_train, y_train)
Y_pred=perceptron.predict(X_test)
train_acc_perceptron=round(perceptron.score(X_train, y_train)*100, 2)
test_acc_perceptron=round(perceptron.score(X_test, y_test)*100, 2)
train_acc_perceptron,test_acc_perceptron


#SDGclassifier --> 과소적합 (max_iter 늘려야됨)
sgd=linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
Y_pred=sgd.predict(X_test)
sgd.score(X_train, y_train)
train_acc_sgd=round(sgd.score(X_train, y_train)*100, 2)
test_acc_sgd=round(sgd.score(X_test, y_test)*100, 2)
train_acc_sgd,test_acc_sgd


#모델 비교 방식
results = pd.DataFrame({
 'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest','Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 'Decision Tree'],
 'train_Score': [train_acc_svc, train_acc_knn, train_acc_log,train_acc_random_forest,
train_acc_gaussian, train_acc_perceptron, train_acc_sgd, train_acc_decision_tree],
 'test_Score': [test_acc_svc, test_acc_knn,test_acc_log, test_acc_random_forest, test_acc_gaussian, test_acc_perceptron, test_acc_sgd, test_acc_decision_tree]})
result_df=results.sort_values(by='train_Score', ascending=False)
result_df=result_df.set_index('Model')
result_df.head(10)


5. 정리

"모든 모델은 도구일 뿐, 문제에 맞게 골라 써야 한다."

직관이 필요한가? → 결정나무

정확도가 중요한가? → 랜덤 포레스트

실시간 데이터인가? → SGD

적은 데이터인가? → 나이브 베이즈

변수 간 복잡한 관계가 있는가? → SVM or 앙상블



항목 & 고려 내용

데이터 크기 --> 작다면 SVM/로지스틱, 크면 SGD/랜덤포레스트

변수 간 관계 --> 독립성 높다면 나이브 베이즈 적합

정형/비정형 데이터 --> 정형이면 대부분 모델 사용 가능, 비정형이면 딥러닝 계열

실행 시간 --> 실시간 처리 필요하면 SGDClassifier

결과 해석 필요 --> 의사결정나무, 로지스틱 회귀

변수 중요도 및 하이퍼파라미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

# 기존 모델 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 변수 중요도 시각화 및 상위 3개 변수 추출
import matplotlib.pyplot as plt
import numpy as np

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# 변수 중요도 상위 3개
top_3_features = [X_train.columns[i] for i in indices[:3]]
print("Top 3 중요 변수:", top_3_features)

# 시각화
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), X_train.columns[indices], rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                           param_grid, scoring='f1', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best Params:", grid_search.best_params_)

# 튜닝된 모델로 예측
best_model = grid_search.best_estimator_
y_pred_before = model.predict(X_test)
y_pred_after = best_model.predict(X_test)

print("Before tuning - Accuracy:", accuracy_score(y_test, y_pred_before))
print("Before tuning - F1 Score:", f1_score(y_test, y_pred_before, average='macro'))

print("After tuning - Accuracy:", accuracy_score(y_test, y_pred_after))
print("After tuning - F1 Score:", f1_score(y_test, y_pred_after, average='macro'))

오버피팅 점검 및 해결방안

In [ ]:
# 훈련 데이터와 검증 데이터 성능 비교
train_acc = best_model.score(X_train, y_train)
test_acc = best_model.score(X_test, y_test)

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)

성능 차이가 클 경우 → 과적합(overfitting) 가능성 있음.

오버피팅 방지 방법 제안
교차 검증 (Cross-validation)

정규화 (Regularization)

변수 개수 축소 / 불필요한 변수 제거

모델 복잡도 제한 (max_depth, min_samples_split 조정 등)

In [ ]:
# Max depth 제한을 통해 모델 복잡도 제한
simple_model = RandomForestClassifier(max_depth=5, random_state=42)
simple_model.fit(X_train, y_train)
y_simple_pred = simple_model.predict(X_test)

print("After applying max_depth=5 - Accuracy:", accuracy_score(y_test, y_simple_pred))
print("After applying max_depth=5 - F1 Score:", f1_score(y_test, y_simple_pred))

종합 해석 및 제안

### 모델 활용 방안
해당 분류 모델은 예측 정확도가 높고 변수 중요도를 해석할 수 있어, 예측이 필요한 실제 업무에 적용 가능하다. 예: 고객 이탈 예측, 질병 분류, 이상 거래 탐지 등

### 성능 향상을 위한 데이터 측면 제안
1. **더 많은 학습 데이터 확보**: 현재 데이터가 제한적일 경우, 추가 수집을 통해 일반화 성능 향상 가능
2. **도메인 특화된 파생변수 생성**: 단순 변수 외에도 업무 지식에 기반한 파생 변수 추가 시 성능 향상 가능
